In [11]:
import ipywidgets as widgets
from IPython.display import display, clear_output, Image, HTML, Javascript
import cv2
import matplotlib.pyplot as plt
import io
import base64
import time
import os
import face_recognition
import dlib
import webbrowser
import subprocess

# LOGOUT SECTION--------------------------------
# Create label for Logout
logout_label = widgets.HTML(value="<h2><span style='font-size: 24px;'>&#x1F44B;</span> See You Again!!! <span style='font-size: 24px;'>&#x1F44B;</span></h2>")

# Create button to logout
logout_button = widgets.Button(
    description="Logout",
    button_style='danger',
    tooltip='Logout',
    icon='sign-out' # (FontAwesome names without the `fa-` prefix)
)

# Function to logout
def logout_function(b):
    
    # Clear username and password text boxes
    username_textbox.value = ''
    password_textbox.value = ''
    
    # Clear login status output
    with login_status_output:
        clear_output()

    # Show the login tab and hide other tabs
    login_ui.layout.display = 'block'
    tabs_container.layout.display = 'none'
    
# Attach event handlers to the buttons
logout_button.on_click(logout_function)

# Define a container for the logout button
logout_content = widgets.VBox([logout_label,logout_button])

# Function to authenticate login++++++++++++++++++++++++++++++++
def authenticate_login(username, password):
    # Replace this with your actual login authentication logic
    if username == "admin" and password == "admin":
        return True
    else:
        return False

# Username and password text boxes
username_textbox = widgets.Text(placeholder="Enter username")
password_textbox = widgets.Password(placeholder="Enter password")

# Login button
login_button = widgets.Button(description="Login", button_style="success")

# Output widget for displaying login status
login_status_output = widgets.Output()

# Function to handle login button click
def handle_login_button_click(b):
    with login_status_output:
        clear_output()
        username = username_textbox.value.strip()
        password = password_textbox.value.strip()
        
        if authenticate_login(username, password):
            print("Login successful! 😀")
            time.sleep(3)
           
            # Show other tabs and hide the login tab 
            login_ui.layout.display = 'none'
            tabs_container.layout.display = 'block'
            
        else:
            print("Invalid username or password. Please try again.")

# Attach event handler to the login button
login_button.on_click(handle_login_button_click)


# FACE Login *****************

def handle_face_login_button_click(b):
    # Load Camera
    cap = cv2.VideoCapture(0)  # Use 0 for default webcam, or adjust as needed
    
    start_time = None
    login_successful = False
    welcome_displayed = False
    welcome_start_time = None

    # Function to load encodings from a folder
    def load_encodings_from_folder(folder_path):
        encodings = {}
        for filename in os.listdir(folder_path):
            name, ext = os.path.splitext(filename)
            if ext.lower() in ['.jpg', '.png', '.jpeg']:
                image_path = os.path.join(folder_path, filename)
                img = cv2.imread(image_path)
                face_encodings = face_recognition.face_encodings(img)
                if face_encodings:
                    encodings[name] = face_encodings[0]  # Assuming only one face per image
        return encodings
    
    # Load known face encodings (you need to prepare this in advance)
    known_encodings = load_encodings_from_folder("images/known/")
    
    # Initialize face detector and predictor
    detector = dlib.get_frontal_face_detector()

    while True:
        ret, frame = cap.read()
    
        # Convert frame to grayscale for face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
        # Detect faces in the frame
        faces = detector(gray)
    
        # List to store matching names
        matching_names = []
        found_name = "Unknown"
    
        # First scan all faces
        for face in faces:
            # Get the face coordinates
            x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()
    
            # Crop the face from the frame
            face_img = frame[y1:y2, x1:x2]
    
            # Check if the face image is not empty
            if not face_img.size:
                continue  # Skip this face if the image is empty
    
            # Resize face image to a fixed size for face recognition (e.g., 128x128)
            face_img_resized = cv2.resize(face_img, (128, 128))
    
            # Encode the face using dlib's face recognition model
            face_encodings = face_recognition.face_encodings(face_img_resized)
            
            if face_encodings:
                # Perform face matching with known encodings
                for name, known_encoding in known_encodings.items():
                    # Compare face encodings
                    match = face_recognition.compare_faces([known_encoding], face_encodings[0])
                    if match[0]:
                        found_name = name
                        break
    
                # Draw rectangle and label on the frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, found_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    
        # After scanning, proceed if there is a matching face and only one face in the frame
        if len(faces) == 1 and found_name != "Unknown":
            if start_time is None:
                start_time = time.time()
            elif time.time() - start_time >= 3:  # Show "Login Successful" after 3 seconds
                login_successful = True
        else:
            # Reset start time if multiple faces detected or no matching face found
            start_time = None
            login_successful = False
    
        # Display "Login Successful" message after 3 seconds and redirect to website if conditions are met
        if login_successful and not welcome_displayed:
            cv2.putText(frame, "Login Successful!!!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            if time.time() - start_time >= 10: #show welcome message
                welcome_displayed = True
                welcome_start_time = time.time()
        if welcome_displayed:
            # Display "Welcome" message for 5 seconds
            cv2.putText(frame, f"Welcome, {found_name}!", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            if time.time() - welcome_start_time >= 5:
                # Show other tabs and hide the login tab 
                login_ui.layout.display = 'none'
                tabs_container.layout.display = 'block'
                break
    
        # Display the frame
        cv2.imshow("Face Login through Webcam", frame)
    
        # Break the loop on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release the video capture object and destroy windows
    cap.release()
    cv2.destroyAllWindows()

face_login_button = widgets.Button(description="", button_style="primary", icon="fingerprint")
face_login_button.layout.width = '50px'
face_login_button.layout.height = '28px'

# Attach event handler to the FACE login button
face_login_button.on_click(handle_face_login_button_click)


# Arrange widgets in a VBox layout
login_ui = widgets.VBox([
    widgets.HTML(value="<h3>Welcome to Login Panel</h3>"),
    widgets.HBox([widgets.Label(value="Username:"), username_textbox]),
    widgets.HBox([widgets.Label(value="Password :"), password_textbox]),
    widgets.HBox([login_button,face_login_button]),
    login_status_output
])


# Create tabs for image, video, and webcam-------------------+++++++++++++
tab_contents = ['Logout']
tabs = widgets.Tab()
tabs.children = [logout_content]
for i, tab_name in enumerate(tab_contents):
    tabs.set_title(i, tab_name)


# Set height of the entire tab widget
tabs_container = widgets.VBox([tabs], layout={'height': '1000px'})

# Display the login UI
display(login_ui)

# Display the tabs
display(tabs_container)

# Initially, only show the login tab
tabs_container.layout.display = 'none'